# MNRAS Latex Author List Builder

### Josh Dillon, 12/26/19

The following notebook was used to build the author list for *Redundant-Baseline Calibration of the Hydrogen Epoch of Reionization Array* (Dillon et al. 2020) and is meant for MNRAS.



In [1]:
from builders import Authors

In [2]:
author_epoch = Authors(epoch='H1C') # This is the relevant epoch for the data being used. Get the alphabetical author list from here.
affil_epoch = Authors(epoch='H3C')  # This is the most up-to-date affiliations. Look here first for affiliations
author_epoch.getAuthors()
affil_epoch.getAuthors()

Reading builders_H1C.txt
Reading builders_H3C.txt


In [3]:
# Page header author list
short_author_list = 'J.\ S.\ Dillon et al.'

# First authors and primary contributors in order. These are removed from the alphabetical section
nonalpha_authors = ['dillonjoshua', 'leemax', 'alizaki', 'parsonsaaron', 'orosznaomi', 'nunhokeechuneeta', 'la~plantepaul', 'beardsleyadam', 'kernnicholas']

# Corresponding authors
coresponding_author = {'dillonjoshua': 'jsdillon@berkeley.edu'}

In [4]:
# Authors not in either author_epoch or affil_epoch
outside_authors = {'leemax': {'first': 'Max',
                              'middle': '',
                              'last': 'Lee',
                              'nice': ['Max', '', 'Lee'],
                              'affiliations': ['Department of Astronomy, University of California, Berkeley, CA'],
                              'group': '_',
                              'ID': {}},
                  'orosznaomi': {'first': 'Naomi',
                                 'middle': '',
                                 'last': 'Orosz',
                                 'nice': ['Naomi', '', 'Orosz'],
                                 'affiliations': ['Department of Astronomy, University of California, Berkeley, CA'],
                                 'group': '_',
                                 'ID': {}},
                  }

In [5]:
# Add authors in the affiliation epoch but not the author epoch
affil_authors_to_add_alphabetically = ['bullphil','santosmario', 'murraysteven', 'pascuarobert']
for a in affil_authors_to_add_alphabetically:
    author_epoch.authors[a] = affil_epoch.authors[a]
    
H0C_authors = ['tegmarkmax']
H0C_epoch = Authors(epoch='H0C')
H0C_epoch.getAuthors()
for a in H0C_authors:
    author_epoch.authors[a] = H0C_epoch.authors[a]


Reading builders_H0C.txt


In [6]:
# Used for fellowship affiliations, etc.
special_affil_marks = [r'\dagger', r'\ddagger', r'\dagger\dagger', r'\ddagger\ddagger']
special_affils = {'dillonjoshua': 'NSF Astronomy and Astrophysics Postdoctoral Fellow',
                  'beardsleyadam': 'NSF Astronomy and Astrophysics Postdoctoral Fellow',
                  'thyagarajannithyanandan': 'NRAO Jansky Fellow'
                 }

In [7]:
# Vague attempt to put \newauthor commands in the right place. Doesn't account for character width, so this might need to be adjusted by hand later.
line_width = 80

In [8]:
tex = r'\author[' + short_author_list + ']{'
affil_counter = 1
special_affil_counter = 1
authors = nonalpha_authors + sorted([author for author in author_epoch.authors if author not in nonalpha_authors])
affil_to_num = {}
special_affil_to_symbol = {}

line_chars = 0
for a in authors:
    entry_chars = 0
    
    # find the author and their affiliation
    try:
        try:
            adict = author_epoch.authors[a]
        except:
            adict = affil_epoch.authors[a]
        try: 
            affiliations = affil_epoch.authors[a]['affiliations']
        except:
            affiliations = author_epoch.authors[a]['affiliations']
    except:
        adict = outside_authors[a]
        affiliations = adict['affiliations']
        
    # Build the author's name
    entry = adict['first']
    if adict['middle'] != '':
        entry += '~' +  adict['middle']
    entry += '~' +  adict['last']
    entry_chars += len(entry)

    # Account for institutional numbering 
    for affil in affiliations:
        if 'fellow' not in affil.lower():
            if affil not in affil_to_num:
                affil_to_num[affil] = str(affil_counter)
                affil_counter += 1
    affil_nums = [affil_to_num[affil] for affil in affiliations if 'fellow' not in affil.lower()]
    entry_chars += len(','.join(affil_nums))
    
    # Account for special affiliations with symbols
    if a in special_affils:
        if special_affils[a] not in special_affil_to_symbol:
            special_affil_to_symbol[special_affils[a]] = special_affil_marks[special_affil_counter-1]
            special_affil_counter += 1
        affil_nums.append(special_affil_to_symbol[special_affils[a]])
        entry_chars += 2
    
    # Add superscript affiliations
    entry += r'$^{' + ','.join(affil_nums) + r'}$'
        
    # Handle corresponding authors \thanks
    if a in coresponding_author:
        entry += r'\thanks{Email: ' + coresponding_author[a] + r'}'
        entry_chars += 1

    # Add comma seperator
    if a != authors[-1]:
        entry += ',\n'
        entry_chars += 2
    
    # try to figure out whether we need a new line in the author list
    if line_chars + entry_chars > line_width:
        entry = r'\newauthor' + '\n' + entry
        line_chars = entry_chars
    else:
        line_chars += entry_chars
    
    # update tex string
    tex += entry

# separate authors and affiliations
tex += '\n' + r'\\' + '\n'

# enumerate affilaitions
for affil, num in affil_to_num.items():
    tex += '$^{' + num + '}$ ' + affil.replace('&', r'\&').replace(' |',',') + r'\\' + '\n'
for special_affil, symbol in special_affil_to_symbol.items():
    tex += '$^{' + symbol + '}$ ' + special_affil.replace('&', r'\&').replace(' |',',') + r'\\' + '\n'

# print final result    
tex = tex[:-3] + '\n}'
print(tex)
    

\author[J.\ S.\ Dillon et al.]{Joshua~S.~Dillon$^{1,\dagger}$\thanks{Email: jsdillon@berkeley.edu},
Max~Lee$^{1}$,
Zaki~S.~Ali$^{1}$,
Aaron~R.~Parsons$^{1}$,
Naomi~Orosz$^{1}$,
\newauthor
Chuneeta~Devi~Nunhokee$^{1}$,
Paul~La~Plante$^{1,2}$,
Adam~P.~Beardsley$^{3,\dagger}$,
\newauthor
Nicholas~S.~Kern$^{1}$,
Zara~Abdurashidova$^{1}$,
James~E.~Aguirre$^{2}$,
Paul~Alexander$^{4}$,
\newauthor
Yanga~Balfour$^{5}$,
Gianni~Bernardi$^{6,7,5}$,
Tashalee~S.~Billings$^{2}$,
Judd~D.~Bowman$^{3}$,
\newauthor
Richard~F.~Bradley$^{8}$,
Phil~Bull$^{9}$,
Jacob~Burba$^{10}$,
Steve~Carey$^{4}$,
\newauthor
Chris~L.~Carilli$^{11}$,
Carina~Cheng$^{1}$,
David~R.~DeBoer$^{1}$,
Matt~Dexter$^{1}$,
\newauthor
Eloy~de~Lera~Acedo$^{4}$,
John~Ely$^{4}$,
Aaron~Ewall-Wice$^{12}$,
Nicolas~Fagnoni$^{4}$,
\newauthor
Randall~Fritz$^{5}$,
Steven~R.~Furlanetto$^{13}$,
Kingsley~Gale-Sides$^{4}$,
\newauthor
Brian~Glendenning$^{11}$,
Deepthi~Gorthi$^{1}$,
Bradley~Greig$^{14}$,
Jasper~Grobbelaar$^{5}$,
\newauthor
Ziyaad~Halda